# Credit Card Fraud Detection

Introduction:

Imagine a world where every swipe of a credit card tells a story—a story of trust, transactions, and the ever-present threat of fraud. This dataset opens a window into that world, focusing on credit card transactions made by European cardholders in September 2013. It's not just about numbers; it's about the pulse of financial security in an increasingly digital age.

Within these numbers lie tales of resilience and vulnerability. Among over a quarter-million transactions, there were 492 instances of fraud, highlighting the constant battle against deceptive practices. This dataset, though numerical in nature due to confidentiality constraints, speaks volumes about the challenges faced by credit card companies in detecting and preventing fraud.

The heartbeat of this dataset is the imbalance—it's a reminder that detecting fraud isn't just about accuracy but about understanding the nuances of class imbalance. With features derived from PCA (Principal Component Analysis) and insights into transaction amounts and timing, this data journey isn't just about predicting fraud; it's about understanding the intricate dance between security and convenience in our financial lives.

As we delve into these numbers, we invite you to explore the strategies, methodologies, and lessons learned in the ongoing quest to safeguard financial transactions. Let's uncover the hidden stories behind each transaction, where vigilance meets innovation in the realm of credit card fraud detection.

In [ ]:
import pandas as pd
import numpy as np
import time
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()

In [ ]:
credit_card_data = pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

## Pre-Processing

In [ ]:
    # Shuffle and randomize data
shuffled_data = credit_card_data.sample(frac=1)
    # Change Class column into Class_0 ([1 0] for legit data) and Class_1 ([0 1] for fraudulent data)
one_hot_data = pd.get_dummies(shuffled_data, columns=['Class'])
one_hot_data['Class_0'] = one_hot_data['Class_0'].replace({True: 1, False: 0})
one_hot_data['Class_1'] = one_hot_data['Class_1'].replace({True: 1, False: 0})
    # Change all values into numbers between 0 and 1
normalized_data = (one_hot_data - one_hot_data.min()) / (one_hot_data.max() - one_hot_data.min())
    # Store just columns V1 through V28 in df_X and columns Class_0 and Class_1 in df_y
df_X = normalized_data.drop(['Class_0', 'Class_1'], axis=1)
df_y = normalized_data[['Class_0', 'Class_1']]
    # Convert both data_frames into np arrays of float32
ar_X, ar_y = np.asarray(df_X.values, dtype='float32'), np.asarray(df_y.values, dtype='float32')

Train-Test-Split

In [ ]:
    # Allocate first 80% of data into training data and remaining 20% into testing data
train_size = int(0.8 * len(ar_X))
(raw_X_train, raw_y_train) = (ar_X[:train_size], ar_y[:train_size]) # first 80% TRAIN
(raw_X_test, raw_y_test) = (ar_X[train_size:], ar_y[train_size:]) # last 20% TEST

Adjusting Weight

In [ ]:
    # Gets a percent of fraud vs legit transactions (0.0017% of transactions are fraudulent)
count_legit, count_fraud = np.unique(credit_card_data['Class'], return_counts=True)[1]
fraud_ratio = float(count_fraud / (count_legit + count_fraud))
print('Percent of fraudulent transactions: ', fraud_ratio)

In [ ]:
#Applies a logit weighting of 578 (1/0.0017) to fraudulent transactions to cause model to pay more attention to them
weighting = 1 / fraud_ratio
raw_y_train[:, 1] = raw_y_train[:, 1] * weighting # select positive samples and apply this operation

## Variables

Defining Dimentions of layers

In [ ]:
# 30 cells for the input
input_dimensions = ar_X.shape[1]
# 2 cells for the output
output_dimensions = ar_y.shape[1]
# 100 cells for the 1st layer
num_layer_1_cells = 100
# 150 cells for the second layer
num_layer_2_cells = 150
num_epochs = 100

Defining Variables to be used later

In [ ]:
    # We will use these as inputs to the model when it comes time to train it (assign values at run time)
X_train_node = tf.placeholder(tf.float32, [None, input_dimensions], name='X_train')
y_train_node = tf.placeholder(tf.float32, [None, output_dimensions], name='y_train')

X_test_node = tf.constant(raw_X_test, name='X_test')
y_test_node = tf.constant(raw_y_test, name='y_test')

# First layer takes in input and passes output to 2nd layer
weight_1_node = tf.Variable(tf.zeros([input_dimensions, num_layer_1_cells]), name='weight_1')
biases_1_node = tf.Variable(tf.zeros([num_layer_1_cells]), name='biases_1')

# Second layer takes in input from 1st layer and passes output to 3rd layer
weight_2_node = tf.Variable(tf.zeros([num_layer_1_cells, num_layer_2_cells]), name='weight_2')
biases_2_node = tf.Variable(tf.zeros([num_layer_2_cells]), name='biases_2')

# Third layer takes in input from 2nd layer and outputs [1 0] or [0 1] depending on fraud vs legit
weight_3_node = tf.Variable(tf.zeros([num_layer_2_cells, output_dimensions]), name='weight_3')
biases_3_node = tf.Variable(tf.zeros([output_dimensions]), name='biases_3')

## Functions

Model's Network Function

In [ ]:
# Function to run an input tensor through the 3 layers and output a tensor that will give us a fraud/legit result
# Each layer uses a different function to fit lines through the data and predict whether a given input tensor will
# result in a fraudulent or legitimate transaction
def network(input_tensor):
    # Sigmoid fits modified data well
    layer1 = tf.nn.sigmoid(tf.matmul(input_tensor, weight_1_node) + biases_1_node)
    # Dropout prevents model from becoming lazy and over confident
    layer2 = tf.nn.dropout(tf.nn.sigmoid(tf.matmul(layer1, weight_2_node) + biases_2_node), 0.85)
    # Softmax works very well with one hot encoding which is how results are outputted
    layer3 = tf.nn.softmax(tf.matmul(layer2, weight_3_node) + biases_3_node)
    return layer3

In [ ]:
# Used to predict what results will be given training or testing input data
# Remember, X_train_node is just a placeholder for now. We will enter values at run time
y_train_prediction = network(X_train_node)
y_test_prediction = network(X_test_node)

Loss function

In [ ]:
    # Cross entropy loss function measures differences between actual output and predicted output
cross_entropy = tf.losses.softmax_cross_entropy(y_train_node, y_train_prediction)

Using a learning rate of 0.005, Adam Optimizer modifies layer 3 variables in an attempt to reduce the loss function.

In [ ]:
optimizer = tf.train.AdamOptimizer(0.005).minimize(cross_entropy)

Function to calculate accuracy

In [ ]:
def calculate_accuracy(actual, predicted):
    actual = np.argmax(actual, 1)
    predicted = np.argmax(predicted, 1)
    return (100 * np.sum(np.equal(predicted, actual)) / predicted.shape[0])

## Tensorflow Session

In [ ]:
with tf.Session() as session:
    tf.global_variables_initializer().run()
    for epoch in range(num_epochs):

        start_time = time.time()

        _, cross_entropy_score = session.run([optimizer, cross_entropy],
                                             feed_dict={X_train_node: raw_X_train, y_train_node: raw_y_train})

        if epoch % 10 == 0:
            timer = time.time() - start_time

            print('Epoch: {}'.format(epoch), 'Current loss: {0:.4f}'.format(cross_entropy_score),
                  'Elapsed time: {0:.2f} seconds'.format(timer))

            final_y_test = y_test_node.eval()
            final_y_test_prediction = y_test_prediction.eval()
            final_accuracy = calculate_accuracy(final_y_test, final_y_test_prediction)
            print("Current accuracy: {0:.2f}%".format(final_accuracy))

    final_y_test = y_test_node.eval()
    final_y_test_prediction = y_test_prediction.eval()
    final_accuracy = calculate_accuracy(final_y_test, final_y_test_prediction)
    print("Final accuracy: {0:.2f}%".format(final_accuracy))

## Final Accuracy

In [ ]:
final_fraud_y_test = final_y_test[final_y_test[:, 1] == 1]
final_fraud_y_test_prediction = final_y_test_prediction[final_y_test[:, 1] == 1]
final_fraud_accuracy = calculate_accuracy(final_fraud_y_test, final_fraud_y_test_prediction)
print('Final fraud specific accuracy: {0:.2f}%'.format(final_fraud_accuracy))

Hope you enjoyed